In [1]:
%run Format.ipynb
import ROOT as root
from array import array
from math import sin
from ROOT import gRandom
from ROOT import TFile
from ROOT import TPad, TLegend
%jsroot on

Welcome to JupyROOT 6.29/01


In [2]:
isigma = 2
N_centr =5
N_hists = 6
centralities = ["0-88%","0-20%","20-40%","40-60%","60-88%"]
file_path="input/"
file_names="se-merge_18945_HeAu_Artem_raw_dphis.root"
histo_names = ["dphi_pt_cent_distr","dphi_min_pt_cent_distr","dphi_cnt_pt_cent_distr","dz_pt_cent_distr","dz_min_pt_cent_distr","dz_cnt_pt_cent_distr"]

In [3]:
hists_emc_read = []
infile = root.TFile.Open(file_path+file_names, "read")

for histname in histo_names:
    hists_emc_read.append(infile.Get(histname))
    hists_emc_read[-1].SetDirectory(root.nullptr)


hist_gamma   = infile.Get("gamma_pt_distr")
hist_gamma.SetDirectory(root.nullptr)

infile.Close()

In [4]:
centrality = [1,-1,1,2,3,4,5,6,7,9]

pt = array( 'd' )
pt_boarders = array( 'd', [0.,.5,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0,6.0,8.0,10.,20.])
pt_err = array( 'd' )
N_pt = len(pt_boarders) - 1
for i in range(N_pt):
    pt.append( (pt_boarders[i]+pt_boarders[i+1])/2)
    pt_err.append(0)
pt_boarders.append(20.)
print(pt,"\n",pt_boarders)

array('d', [0.25, 0.75, 1.25, 1.75, 2.25, 2.75, 3.25, 3.75, 4.25, 4.75, 5.5, 7.0, 9.0, 15.0]) 
 array('d', [0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 6.0, 8.0, 10.0, 20.0, 20.0])


In [5]:
boards_tot = [[-0.1,0.1],[-50,50]]
boards_fit = [[-0.02,0.02],[-8,8]]

def emc_fitter(InHist, centr, seed=1, sigma=2, fits_mean=0, fits_sigma=0):
    N_gaus=1
    seed0 = seed
    seed=seed*N_pt
    print(pt)

    hist_pt_bin = InHist.ProjectionY("kek")

    Mean_local =  array( 'd' )
    Sigma_local =  array( 'd' )
    Mean_local_err =  array( 'd' )
    Sigma_local_err =  array( 'd' )

    int_fg_local =  array( 'd' )
    int_bg_local =  array( 'd' )
 
    for p in range(N_pt):
        c1 = root.TCanvas(f"c1{seed+p}",f"c1{seed+p}",720,360)

        emc_var_porj = InHist.ProjectionX(f"emc{seed+p}",hist_pt_bin.FindBin(pt_boarders[p]),hist_pt_bin.FindBin(pt_boarders[p+1])-1,centrality[2*centr],centrality[2*centr+1])
        print("CENTRLITY :", centrality[2*centr], "-",centrality[2*centr+1])
        print("PT :", pt_boarders[p], "-",pt_boarders[p+1])
        print("bins :", pt_boarders[p], "-",pt_boarders[p+1])
        emc_var_porj.SetLineColor(1)

        par_tot = array( 'd', (3*N_gaus+3)*[2.] )
        g_gaus= root.TF1( f'g_gaus{seed+p}', 'gaus',  boards_fit[int(seed0/3)][0], boards_fit[int(seed0/3)][1])
        g_pol = root.TF1( f'g_pol{seed+p}', '[0]+[1]*x*(abs(x)/x-1)+[2]*x*(abs(x)/x+1)',  boards_tot[int(seed0/3)][0],boards_tot[int(seed0/3)][1])
        Format_Func(g_gaus,3,4)
        Format_Func(g_pol,3,2)

        total = root.TF1( f'total_pos{seed+p}', 'gaus(0)+[3]+[4]*x*(abs(x)/x-1)+[5]*x*(abs(x)/x+1)',boards_tot[int(seed0/3)][0], boards_tot[int(seed0/3)][1])
        Format_Func(total,2,1)


        if fits_mean!=0 and fits_sigma!=0:
            mean_ii=fits_sigma.Eval(pt[p])
            sigma_ii=fits_sigma.Eval(pt[p])
            g_gaus.SetParameter(1,sigma_ii)
            g_gaus.SetParameter(2,sigma_ii)
            g_gaus.SetParLimits(1,mean_ii*0.8-0.05,mean_ii*1.2+0.05)
            g_gaus.SetParLimits(2,sigma_ii*0.5,sigma_ii*1.5)
        else:
            g_gaus.SetParameter(1,0.)
            g_gaus.SetParameter(2,boards_fit[int(seed0/3)][1])


        #emc_var_porj.GetXaxis().SetRangeUser(-1.4, 0.5);
        emc_var_porj.Draw()
        emc_var_porj.Fit( g_gaus, 'QR')
        emc_var_porj.Fit( g_pol, 'QR+' )

        par_gaus = g_gaus.GetParameters()
        par_pol = g_pol.GetParameters()

        par_tot[0], par_tot[1], par_tot[2], par_tot[3], par_tot[4], par_tot[5] = par_gaus[0], par_gaus[1], par_gaus[2], par_pol[0], par_pol[1], par_pol[1]

        total.SetParameters( par_tot )
        total.SetParLimits(1,boards_tot[int(seed0/3)][0]/10,boards_tot[int(seed0/3)][1]/10)
        total.SetParLimits(2,boards_fit[int(seed0/3)][1]/10,boards_tot[int(seed0/3)][1]/2)
        
        emc_var_porj.Draw()
        emc_var_porj.Fit( total, 'R' )

        par_tot = total.GetParameters()

        par_gaus[0], par_gaus[1], par_gaus[2], par_pol[0], par_pol[1], par_pol[2] = par_tot[0], par_tot[1], par_tot[2], par_tot[3], par_tot[4], par_tot[5]

        g_gaus.SetParameters(par_gaus)
        g_pol.SetParameters(par_pol)

        g_gaus.Draw('same')
        g_pol.Draw('same')

        Mean_local.append(total.GetParameter(1))
        Sigma_local.append(total.GetParameter(2))
        Mean_local_err.append(total.GetParError(1))
        Sigma_local_err.append(total.GetParError(2))

        int_fg_local.append(g_gaus.Integral(total.GetParameter(1)-sigma*total.GetParameter(2),total.GetParameter(1)+sigma*total.GetParameter(2)))
        int_bg_local.append(g_pol.Integral(total.GetParameter(1)-sigma*total.GetParameter(2),total.GetParameter(1)+sigma*total.GetParameter(2)))
            
        #m2_pos.Draw("same") m2_neg.Draw("same")
        m2_name=str(seed0)+"/"+str(centr)+"/emc_"+str(round(pt[p],1))

        if fits_mean==0:
            c1.SaveAs( "output/" + m2_name + ".png" )
        else:
            c1.SaveAs( "output/" + m2_name + "upd.png" )

    return [Mean_local, Mean_local_err, Sigma_local, Sigma_local_err, int_fg_local, int_bg_local]


In [6]:
def emc_centr_fitter(InHist, centralities, seed=1, sigma=2, fits_mean=0, fits_sigma=0):
    hist_ser = []
    for centr in centralities:
        hist_ser.append(emc_fitter(InHist, centr, seed, sigma, fits_mean, fits_sigma))
    return hist_ser

In [7]:
import multiprocess as mp

Ntr = N_hists

print([(hists_emc_read[i].GetName(), 0, i) for i in range(Ntr)])

pool = mp.Pool(Ntr)
output_array = pool.starmap(emc_centr_fitter, [(hists_emc_read[i], [0,1,2,3,4], i, isigma) for i in range(Ntr)])
pool.close()

[('dphi_pt_cent_distr', 0, 0), ('dphi_min_pt_cent_distr', 0, 1), ('dphi_cnt_pt_cent_distr', 0, 2), ('dz_pt_cent_distr', 0, 3), ('dz_min_pt_cent_distr', 0, 4), ('dz_cnt_pt_cent_distr', 0, 5)]


array('d', [0.25, 0.75, 1.25, 1.75, 2.25, 2.75, 3.25, 3.75, 4.25, 4.75, 5.5, 7.0, 9.0, 15.0])array('d', [0.25, 0.75, 1.25, 1.75, 2.25, 2.75, 3.25, 3.75, 4.25, 4.75, 5.5, 7.0, 9.0, 15.0])

array('d', [0.25, 0.75, 1.25, 1.75, 2.25, 2.75, 3.25, 3.75, 4.25, 4.75, 5.5, 7.0, 9.0, 15.0])
array('d', [0.25, 0.75, 1.25, 1.75, 2.25, 2.75, 3.25, 3.75, 4.25, 4.75, 5.5, 7.0, 9.0, 15.0])array('d', [0.25, 0.75, 1.25, 1.75, 2.25, 2.75, 3.25, 3.75, 4.25, 4.75, 5.5, 7.0, 9.0, 15.0])

array('d', [0.25, 0.75, 1.25, 1.75, 2.25, 2.75, 3.25, 3.75, 4.25, 4.75, 5.5, 7.0, 9.0, 15.0])
CENTRLITY : 1 CENTRLITY :-  1-1 -
 -1PT :
 CENTRLITY :0.0  PT :-1  CENTRLITY : 0.50.0 CENTRLITY :CENTRLITY :-1
   bins :  11--1-    
-0.0 -  0.5PT :-1-
  
-10.5 -1PT :


bins : PT :PT :0.0  0.00.0 0.0  -  - -0.00.5  - 0.5
0.5 -
0.5bins :bins :

  bins :0.50.0   0.0
-0.0 bins :  - 0.5 -0.0
 0.50.5 

- 0.5
CENTRLITY : 1 CENTRLITY :-  1CENTRLITY :-1
  PT :1  0.5- - -1 - 1.0-1

PT :
bins :PT :  0.5  - 0.50.51.0 
 bins :--  1.0 0.5
 bins

Info in <TCanvas::Print>: png file output/5/0/emc_0.2.png has been created
Info in <TCanvas::Print>: png file output/3/0/emc_0.2.png has been created
Info in <TCanvas::Print>: png file output/4/0/emc_0.2.png has been created
Info in <TCanvas::Print>: png file output/0/0/emc_0.2.png has been created
Info in <TCanvas::Print>: png file output/2/0/emc_0.2.png has been created
Info in <TCanvas::Print>: png file output/5/0/emc_0.8.png has been created
Info in <TCanvas::Print>: png file output/3/0/emc_0.8.png has been created
Info in <TCanvas::Print>: png file output/4/0/emc_0.8.png has been created
Info in <TCanvas::Print>: png file output/1/0/emc_0.2.png has been created
Info in <TCanvas::Print>: png file output/0/0/emc_0.8.png has been created
Info in <TCanvas::Print>: png file output/2/0/emc_0.8.png has been created
Info in <TCanvas::Print>: png file output/5/0/emc_1.2.png has been created
Info in <TCanvas::Print>: png file output/4/0/emc_1.2.png has been created
Info in <TCanvas::Print>:

In [8]:
Means, Mean_errs, Sigmas, Sigma_errs, Int_FG, Int_BG = [], [], [], [], [], []
for i in range(Ntr):
    Means0, Mean_errs0, Sigmas0, Sigma_errs0, Int_FG0, Int_BG0 = [], [], [], [], [], []
    for j in range(N_centr):
        Means0.append(output_array[i][j][0])
        Mean_errs0.append(output_array[i][j][1])
        Sigmas0.append(output_array[i][j][2])
        Sigma_errs0.append(output_array[i][j][3])
        Int_FG0.append(output_array[i][j][4])
        Int_BG0.append(output_array[i][j][5])
        for ier in range(len(Sigma_errs0[j])):
            Sigma_errs0[j][ier]+=Sigmas0[j][ier]*0.05
            Mean_errs0[j][ier]+=abs(Means0[j][ier])*0.10
    Means.append(Means0)
    Mean_errs.append(Mean_errs0)
    Sigmas.append(Sigmas0)
    Sigma_errs.append(Sigma_errs0)
    Int_FG.append(Int_FG0)
    Int_BG.append(Int_BG0)
print(Sigmas[1])
print(len(Sigmas))

[array('d', [0.006483209911374798, 0.006332574896541319, 0.004979069546025846, 0.004439861612722177, 0.003969714761999189, 0.003689866560623316, 0.0035596998179552386, 0.003285840372634647, 0.003542179716702062, 0.0030181061874788072, 0.003092653924332947, 0.0035967199416764746, 0.002981497158173277, 0.0020000005415103006]), array('d', [0.00637165691515147, 0.006202219336608861, 0.004843989066731175, 0.004254779912741953, 0.0038610241455954686, 0.003641677419739596, 0.003439880775149612, 0.003133921992680277, 0.003841613053105471, 0.0025581022528432806, 0.004561080383724452, 0.003166819064145451, 0.0020000000004966106, 0.008386895453723692]), array('d', [0.006435942706244441, 0.006286775244716753, 0.004947648308239569, 0.0044460646231773105, 0.003918476414284555, 0.003629046480006018, 0.003585946391184894, 0.0032383358155996076, 0.0034700850580205076, 0.0027105680925950796, 0.0023949485330350653, 0.0032692802448106833, 0.0020000001303709007, 0.012862514300775523]), array('d', [0.006554

In [9]:
with open('output/Mean.txt', 'w') as f:
#Mean.Write()
#Sigma.Write()
    for mean_centr in Means:
        for mean_part in mean_centr:
            f.write(f"{mean_part}\n")

with open('output/Sigma.txt', 'w') as f:
#Mean.Write()
#Sigma.Write()
    for sigma_centr in Sigmas:
        for sigma_part in sigma_centr:
            f.write(f"{sigma_part}\n")

In [10]:
colors=[1,2,4,6,root.kGreen+2]
fit_functions = []
for itype in range(N_hists):
    fit_functions_loc = []
    for icentr in range(N_centr):
        #fit_functions_loc.append(root.TF1(f"fit_functions{itype}_{icentr}","(abs(x-[7])/(x-[7])+1)*([0]+[1]/x+[2]/x/x)+(abs(x-[7])/(x-[7])-1)*([4]+[5]*x+[6]*x*x)-(abs(x-4)/(x-4)+1)*([8]+[1]/x+[2]/x/x)"))
        if itype!=2:
            fit_functions_loc.append(root.TF1(f"fit_functions{itype}_{icentr}","[0]+pow(([1]+[5]*x)/(1+pow([2]*x,[3])),[4])"))
        else:
            fit_functions_loc.append(root.TF1(f"fit_1332mean_functions{itype}_{icentr}","([0]+[1]*x+[2]*x*x+[3]*x*x*x+[4]*x*x*x*x+[5]*x*x*x*x*x)*exp(-[6]*x)"))
        if itype<2:
            fit_functions_loc[-1].SetParameter(0,0.00309)
            fit_functions_loc[-1].SetParameter(1,0.0034)
            fit_functions_loc[-1].SetParameter(2,1.13)
            fit_functions_loc[-1].SetParameter(3,6.45)
            fit_functions_loc[-1].SetParameter(4,0.235)
            fit_functions_loc[-1].SetParameter(5,0.0)
            fit_functions_loc[-1].SetLineWidth(3)
        elif itype>2:
            fit_functions_loc[-1].SetParameter(0,2.6)
            fit_functions_loc[-1].SetParameter(1,1.5)
            fit_functions_loc[-1].SetParameter(2,1.2)
            fit_functions_loc[-1].SetParameter(3,1.2)
            fit_functions_loc[-1].SetParameter(4,1.23)
            fit_functions_loc[-1].SetLineWidth(3)
        else: 
            fit_functions_loc[-1].FixParameter(6,0.3)
        fit_functions_loc[-1].SetLineColor(colors[icentr])
    fit_functions.append(fit_functions_loc)
print(fit_functions)

[[<cppyy.gbl.TF1 object at 0x5641bb0451b0>, <cppyy.gbl.TF1 object at 0x5641c0e7b240>, <cppyy.gbl.TF1 object at 0x5641c38e5820>, <cppyy.gbl.TF1 object at 0x5641c34fa4a0>, <cppyy.gbl.TF1 object at 0x5641c33e2c10>], [<cppyy.gbl.TF1 object at 0x5641c2618c50>, <cppyy.gbl.TF1 object at 0x5641bc3f24e0>, <cppyy.gbl.TF1 object at 0x5641c2d1b950>, <cppyy.gbl.TF1 object at 0x5641c38eab70>, <cppyy.gbl.TF1 object at 0x5641c39862c0>], [<cppyy.gbl.TF1 object at 0x5641c399b1e0>, <cppyy.gbl.TF1 object at 0x5641c3922360>, <cppyy.gbl.TF1 object at 0x5641c3920fd0>, <cppyy.gbl.TF1 object at 0x5641c398f7f0>, <cppyy.gbl.TF1 object at 0x5641c3911930>], [<cppyy.gbl.TF1 object at 0x5641c39c1f30>, <cppyy.gbl.TF1 object at 0x5641c3911710>, <cppyy.gbl.TF1 object at 0x5641c39d0d00>, <cppyy.gbl.TF1 object at 0x5641c3975190>, <cppyy.gbl.TF1 object at 0x5641c39225c0>], [<cppyy.gbl.TF1 object at 0x5641c39ab090>, <cppyy.gbl.TF1 object at 0x5641c39156b0>, <cppyy.gbl.TF1 object at 0x5641c39cdc90>, <cppyy.gbl.TF1 object at

In [11]:
Sigmas[3][4][11] = 2.2
Sigma_errs[3][4][11] = 0.01
Sigmas[3][1][9] = 3.2
Sigmas[3][1][13] = 3.5

In [12]:
h2=[]
c3 = root.TCanvas(f"c3",f"c3",500*3,500*2)
c3.Divide(3,2)
g_sigma = [[[]]*N_centr]*N_hists
g_sigma_inter = [[[]]*6]*N_hists
print(g_sigma)
maxes = [0.01,0.01,0.02,5,5,10]
names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
legends =[]
for itype in range(N_hists):
    c3.cd(itype+1)
    h2.append(Format_Pad_old( 0.05, 16, 0, maxes[itype], "p_{T} (GeV/#it{c})", "Sigmas", 1.1, 1.1, 0.03, 0.03, "" ))
    h2[-1].Draw()
    legends.append(root.TLegend(0.3,0.5,0.7,0.9,names[itype]))
    legends[-1].SetFillColorAlpha(0,0)
    legends[-1].SetLineWidth(0)
    for icentr in range(0,N_centr):
        
        g_sigma[itype][icentr].append(root.TGraphErrors(N_pt, pt, Sigmas[itype][icentr],pt_err,Sigma_errs[itype][icentr]))
        g_sigma_inter[itype][icentr].append(root.TGraphErrors(N_pt))
        
        Format_Graph(g_sigma[itype][icentr][-1],20,1, colors[icentr], 3, colors[icentr],1,1)
        Format_Graph(g_sigma_inter[itype][icentr][-1], 21, 0, colors[icentr], 3, colors[icentr], 1, 1)
        for i in range(100):
            g_sigma_inter[itype][icentr][-1].AddPoint(0.5+i*0.1,g_sigma[itype][icentr][-1].Eval(0.5+i*0.1))
        g_sigma[itype][icentr][-1].Fit( fit_functions[itype][icentr],"L","Q")
        #params_mean_sigma[syst][centr][0].append(all_fit_sigma_low[ii].GetParameters())
        g_sigma[itype][icentr][-1].Draw('P')
        #g_sigma_inter[itype][icentr][-1].Draw('sameL')
        legends[-1].AddEntry(g_sigma[itype][icentr][-1],centralities[icentr],"p")
        legends[-1].AddEntry(fit_functions[itype][icentr],f"fit","l")
        
    legends[-1].Draw()
c3.Draw()
c3.SaveAs("output/emc_sigma.png")

[[[], [], [], [], []], [[], [], [], [], []], [[], [], [], [], []], [[], [], [], [], []], [[], [], [], [], []], [[], [], [], [], []]]
 FCN=1.59871 FROM MIGRAD    STATUS=CONVERGED     452 CALLS         453 TOTAL
                     EDM=4.63537e-07    STRATEGY= 1  ERROR MATRIX UNCERTAINTY   2.3 per cent
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           2.91305e-03   4.79852e-04  -1.35152e-06  -3.08389e+00
   2  p1           3.25206e-11   5.41156e-11  -1.71386e-13  -7.58593e+07
   3  p2           1.22807e+00   5.36772e-01   1.29042e-03   8.41574e-04
   4  p3           5.84055e+00   1.77036e+00  -2.87920e-03  -4.56360e-05
   5  p4           2.33901e-01   5.50064e-03  -1.62207e-05  -1.42453e-01
   6  p5           4.34890e-11   1.62980e-10   5.22048e-13  -4.09094e+07
 FCN=2.68143 FROM MIGRAD    STATUS=CALL LIMIT   1793 CALLS        1794 TOTAL
                     EDM=0.000342634  

Warning in <Fit>: Abnormal termination of minimization.
Warning in <Fit>: Abnormal termination of minimization.
Warning in <Fit>: Abnormal termination of minimization.
Warning in <TROOT::Append>: Replacing existing TH1: sec (Potential memory leak).
Warning in <Fit>: Abnormal termination of minimization.
Warning in <Fit>: Abnormal termination of minimization.
Warning in <Fit>: Abnormal termination of minimization.
Warning in <TROOT::Append>: Replacing existing TH1: sec (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: sec (Potential memory leak).
Warning in <Fit>: Abnormal termination of minimization.
Warning in <Fit>: Abnormal termination of minimization.
Warning in <Fit>: Abnormal termination of minimization.
Warning in <Fit>: Abnormal termination of minimization.
Warning in <Fit>: Abnormal termination of minimization.
Warning in <TROOT::Append>: Replacing existing TH1: sec (Potential memory leak).
Warning in <Fit>: Abnormal termination of minimization.
Warn

In [13]:
fit_mean_functions0,  fit_mean_functions = [], []
for itype in range(N_hists):
    fit_mean_functions_loc, fit_mean_functions_loc0 = [], []
    for icentr in range(N_centr):
        fit_mean_functions_loc0.append(root.TF1(f"fit_1mean_functions{itype}_{icentr}","0*([0]*[6])+([1]+[2]*x+[3]*x*x*x+[4]*x*x*x*x+[5]*sqrt(x))",0.25,15.25))
        fit_mean_functions_loc.append(root.TF1(f"fit_1332mean_functions{itype}_{icentr}","exp(-[0]*x)*([1]+[2]*x+[3]*x*x*x+[4]*x*x*x*x+[5]*sqrt(x))+[6]",0.25,15.25))
        fit_mean_functions_loc[-1].SetLineWidth(3)
        fit_mean_functions_loc[-1].SetLineColor(colors[icentr])

    fit_mean_functions.append(fit_mean_functions_loc)
    fit_mean_functions0.append(fit_mean_functions_loc0)
print(fit_mean_functions)

[[<cppyy.gbl.TF1 object at 0x5641c3a73570>, <cppyy.gbl.TF1 object at 0x5641c4842fa0>, <cppyy.gbl.TF1 object at 0x5641c43aa6a0>, <cppyy.gbl.TF1 object at 0x5641c3f98430>, <cppyy.gbl.TF1 object at 0x5641c480cab0>], [<cppyy.gbl.TF1 object at 0x5641c47e4bf0>, <cppyy.gbl.TF1 object at 0x5641c4861e80>, <cppyy.gbl.TF1 object at 0x5641c47ef9e0>, <cppyy.gbl.TF1 object at 0x5641c2f6c720>, <cppyy.gbl.TF1 object at 0x5641c47e6f50>], [<cppyy.gbl.TF1 object at 0x5641c47d2850>, <cppyy.gbl.TF1 object at 0x5641c481ee70>, <cppyy.gbl.TF1 object at 0x5641c4840df0>, <cppyy.gbl.TF1 object at 0x5641c47c4870>, <cppyy.gbl.TF1 object at 0x5641c2f6ec20>], [<cppyy.gbl.TF1 object at 0x5641c47c0a60>, <cppyy.gbl.TF1 object at 0x5641c47c7ca0>, <cppyy.gbl.TF1 object at 0x5641c47f0da0>, <cppyy.gbl.TF1 object at 0x5641c4863670>, <cppyy.gbl.TF1 object at 0x5641c48122f0>], [<cppyy.gbl.TF1 object at 0x5641c47e0d90>, <cppyy.gbl.TF1 object at 0x5641c47da7e0>, <cppyy.gbl.TF1 object at 0x5641c47dd970>, <cppyy.gbl.TF1 object at

In [14]:
h3=[]
c4 = root.TCanvas(f"c4",f"c4",500*3,500*2)
c4.Divide(3,2)
g_mean = [[[]]*N_centr]*N_hists
print(g_mean)
maxes = [0.001,0.001,0.002,1,2,2]
names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
legends =[]
for itype in range(N_hists):
    c4.cd(itype+1)
    h3.append(Format_Pad_old( 0.25, 16, -maxes[itype], maxes[itype], "p_{T} (GeV/#it{c})", "Means", 1.1, 1.1, 0.03, 0.03, "" ))
    h3[-1].Draw()
    legends.append(root.TLegend(0.3,0.5,0.7,0.9,names[itype]))
    legends[-1].SetFillColorAlpha(0,0)
    legends[-1].SetLineWidth(0)
    for icentr in range(0,N_centr):

        g_mean[itype][icentr].append(root.TGraphErrors(N_pt, pt, Means[itype][icentr],pt_err,Sigma_errs[itype][icentr]))
        Format_Graph(g_mean[itype][icentr][-1],20,1, colors[icentr], 3, colors[icentr],1,1)
        g_mean[itype][icentr][-1].AddPoint(0.5,0.5*(Means[itype][icentr][0]+Means[itype][icentr][1]))
        g_mean[itype][icentr][-1].SetPointError(N_pt,0,Sigma_errs[itype][icentr][0])
        g_mean[itype][icentr][-1].Fit(fit_mean_functions0[itype][icentr],"M","Q",0.25,3.25)
        fit_mean_functions[itype][icentr].SetParameters(fit_mean_functions0[itype][icentr].GetParameters())
        fit_mean_functions[itype][icentr].SetParameter(0,2)
        if itype!=2: fit_mean_functions[itype][icentr].FixParameter(6,0)
        g_mean[itype][icentr][-1].Fit(fit_mean_functions[itype][icentr],"M","Q",0.25,15.25)
        #params_mean_sigma[syst][centr][0].append(all_fit_sigma_low[ii].GetParameters())
        g_mean[itype][icentr][-1].Draw('P')
        legends[-1].AddEntry(g_mean[itype][icentr][-1],centralities[icentr],"p")
        legends[-1].AddEntry(fit_mean_functions[itype][icentr],f"fit","l")
        
    legends[-1].Draw()
c4.Draw()
c4.SaveAs("output/emc_mean.png")

[[[], [], [], [], []], [[], [], [], [], []], [[], [], [], [], []], [[], [], [], [], []], [[], [], [], [], []], [[], [], [], [], []]]
 FCN=1.06144 FROM HESSE     STATUS=FAILED         11 CALLS         502 TOTAL
                     EDM=8.85444e-19    STRATEGY= 1  ERROR MATRIX UNCERTAINTY 100.0 per cent
  EXT PARAMETER                APPROXIMATE        STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0          -4.06098e+04   2.00000e+00   1.41421e-02   9.97175e-01
   2  p1          -2.72378e-03   9.14104e-05   2.66833e-05  -9.38895e+05
   3  p2          -8.42728e-03   4.23301e-05   5.01331e-05  -2.45493e+06
   4  p3           8.93968e-04   5.27718e-06   4.63725e-06  -1.88976e+07
   5  p4          -1.72949e-04   1.73331e-06   9.29410e-07  -5.50130e+07
   6  p5           1.03641e-02   6.58364e-05   7.37366e-05  -1.51243e+06
   7  p6           0.00000e+00   1.41421e+00   1.41421e-02  -3.63790e-01
 FCN=2.00228 FROM HESSE     STATUS=OK     

Warning in <TROOT::Append>: Replacing existing TH1: sec (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: sec (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: sec (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: sec (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: sec (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: sec (Potential memory leak).
Warning in <Fit>: Abnormal termination of minimization.
Info in <TCanvas::Print>: png file output/emc_mean.png has been created


In [15]:
bw_dphi_hist = hists_emc_read[1].ProjectionX("keks")
bw_dphi = bw_dphi_hist.GetBinWidth(1)
bw_dz_hist = hists_emc_read[4].ProjectionX("kekz")
bw_dz = bw_dz_hist.GetBinWidth(1)

FG_BG, FG_BG_err = [], []
for icase in range(12):
    FG_BG0, FG_BG_err0 = [], []
    for icentr in range(N_centr):
        hist_gamma_proj = hist_gamma.ProjectionX(f"gamma{icentr}",centrality[2*icentr],centrality[2*icentr+1])
        FG_BG_loc =  array( 'd' )
        FG_BG_err_loc =  array( 'd' )
        for ipt in range(N_pt):
            gamma_yield = 0.1
            for j in range(hist_gamma_proj.FindBin(pt_boarders[ipt]),hist_gamma_proj.FindBin(pt_boarders[ipt+1])):
                gamma_yield += hist_gamma_proj.GetBinContent(j)
                
            if icase == 0:
                FG_BG_loc.append(Int_FG[1][icentr][ipt]/(Int_BG[1][icentr][ipt]+Int_FG[1][icentr][ipt]))
                FG_BG_err_loc.append((abs(Int_FG[1][icentr][ipt]))**0.5/(Int_BG[1][icentr][ipt]+Int_FG[1][icentr][ipt])*bw_dphi**0.5)
            if icase == 1:
                FG_BG_loc.append(Int_FG[4][icentr][ipt]/(Int_BG[4][icentr][ipt]+Int_FG[4][icentr][ipt]))
                FG_BG_err_loc.append((abs(Int_FG[4][icentr][ipt]))**0.5/(Int_BG[4][icentr][ipt]+Int_FG[4][icentr][ipt])*bw_dz**0.5)
            if icase == 2:
                FG_BG_loc.append(1-(1-Int_FG[4][icentr][ipt]/(Int_BG[4][icentr][ipt]+Int_FG[4][icentr][ipt]))*(1-Int_FG[1][icentr][ipt]/(Int_BG[1][icentr][ipt]+Int_FG[1][icentr][ipt])))
                FG_BG_err_loc.append((abs(Int_FG[4][icentr][ipt]))**0.5/(Int_BG[4][icentr][ipt]+Int_FG[4][icentr][ipt])*bw_dz**0.5)
            if icase == 3:
                FG_BG_loc.append((Int_BG[1][icentr][ipt]+Int_FG[1][icentr][ipt])/bw_dphi/gamma_yield)
                FG_BG_err_loc.append((abs(Int_FG[1][icentr][ipt]/bw_dphi))**0.5/gamma_yield)
            if icase == 4:
                FG_BG_loc.append((Int_BG[4][icentr][ipt]+Int_FG[4][icentr][ipt])/bw_dz/gamma_yield)
                FG_BG_err_loc.append((abs(Int_FG[4][icentr][ipt]/bw_dz))**0.5/gamma_yield)
            if icase == 5:
                FG_BG_loc.append(1-(1-(Int_BG[1][icentr][ipt]+Int_FG[1][icentr][ipt])/bw_dz/gamma_yield)*(1-(Int_BG[4][icentr][ipt]+Int_FG[4][icentr][ipt])/bw_dz/gamma_yield))
                FG_BG_err_loc.append((abs(Int_FG[4][icentr][ipt]/bw_dz))**0.5/gamma_yield)
            if icase == 6:
                FG_BG_loc.append((Int_FG[1][icentr][ipt])/bw_dphi/gamma_yield)
                FG_BG_err_loc.append((abs(Int_FG[1][icentr][ipt]/bw_dphi))**0.5/gamma_yield)
            if icase == 7:
                FG_BG_loc.append((Int_FG[4][icentr][ipt])/bw_dz/gamma_yield)
                FG_BG_err_loc.append((abs(Int_FG[4][icentr][ipt]/bw_dz))**0.5/gamma_yield)
            if icase == 8:
                FG_BG_loc.append(1-(1-(Int_FG[1][icentr][ipt])/bw_dz/gamma_yield)*(1-(Int_FG[4][icentr][ipt])/bw_dz/gamma_yield))
                FG_BG_err_loc.append((abs(Int_FG[4][icentr][ipt]/bw_dz))**0.5/gamma_yield)
            if icase == 9:
                FG_BG_loc.append((Int_BG[1][icentr][ipt])/bw_dphi/gamma_yield)
                FG_BG_err_loc.append((abs((Int_BG[1][icentr][ipt]+Int_FG[1][icentr][ipt])/bw_dphi))**0.5/gamma_yield)
            if icase == 10:
                FG_BG_loc.append((Int_BG[4][icentr][ipt])/bw_dz/gamma_yield)
                FG_BG_err_loc.append((abs(Int_BG[4][icentr][ipt]/bw_dz))**0.5/hist_gamma_proj.GetBinContent(ipt+1))
            if icase == 11:
                FG_BG_loc.append(1-(1-(Int_BG[1][icentr][ipt])/bw_dz/gamma_yield)*(1-(Int_BG[4][icentr][ipt])/bw_dz/gamma_yield))
                FG_BG_err_loc.append((abs(Int_BG[4][icentr][ipt]/bw_dz+Int_BG[1][icentr][ipt]/bw_dphi))**0.5/gamma_yield)
        FG_BG0.append(FG_BG_loc)
        FG_BG_err0.append(FG_BG_err_loc)
    FG_BG.append(FG_BG0)
    FG_BG_err.append(FG_BG_err0)

print(FG_BG[0][0])

array('d', [0.7674873588695784, 0.7457382607300552, 0.6962231187875513, 0.6600420552409703, 0.6352809347227265, 0.6240402745425913, 0.6163563466092121, 0.6138923641026677, 0.6084003163341282, 0.5980416818397974, 0.5779275966522149, 0.5866885712453536, 0.572612816418843, -0.3803883371426991])


In [16]:
h5=[]
c5 = root.TCanvas(f"c5",f"c5",500*3,500*4)
c5.Divide(3,4)
g_mean = [[[]]*12]*12
print(g_mean)
maxes = [1,1,1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.2,0.2]
names = ["2#sigma d#phi FG/(FG+BG)","2#sigma dz FG/(FG+BG)","2#sigma both","2#sigma d#phi (FG+BG)/N_{#gamma}","2#sigma dz (FG+BG)/N_{#gamma}","2#sigma both",\
         "2#sigma d#phi FG/N_{#gamma}","2#sigma dz FG/N_{#gamma}","2#sigma both","2#sigma d#phi BG/N_{#gamma}","2#sigma dz BG/N_{#gamma}","2#sigma both"]
legends =[]
for itype in range(12):
    c5.cd(itype+1)
    h5.append(Format_Pad_old( 0.25, 16, 0, maxes[itype], "p_{T} (GeV/#it{c})", "Probability", 1.1, 1.1, 0.03, 0.03, "" ))
    h5[-1].Draw()
    legends.append(root.TLegend(0.4,0.6,0.9,0.9))
    legends[-1].SetFillColorAlpha(0,0)
    legends[-1].SetTextAlign(23)
    legends[-1].SetLineWidth(0)
    legends[-1].SetTextSize(0.05)
    for icentr in range(0,N_centr):

        g_mean[itype][icentr].append(root.TGraphErrors(N_pt, pt, FG_BG[itype][icentr],pt_err,FG_BG_err[itype][icentr]))
        Format_Graph(g_mean[itype][icentr][-1], 20, 1, colors[icentr], 3, colors[icentr], 1, 1)

        g_mean[itype][icentr][-1].Draw('P')
        legends[-1].AddEntry(g_mean[itype][icentr][-1],names[itype]+" "+centralities[icentr],"p")
        
    legends[-1].Draw()
c5.Draw()
c5.SaveAs("output/emc_prob.png")

[[[], [], [], [], [], [], [], [], [], [], [], []], [[], [], [], [], [], [], [], [], [], [], [], []], [[], [], [], [], [], [], [], [], [], [], [], []], [[], [], [], [], [], [], [], [], [], [], [], []], [[], [], [], [], [], [], [], [], [], [], [], []], [[], [], [], [], [], [], [], [], [], [], [], []], [[], [], [], [], [], [], [], [], [], [], [], []], [[], [], [], [], [], [], [], [], [], [], [], []], [[], [], [], [], [], [], [], [], [], [], [], []], [[], [], [], [], [], [], [], [], [], [], [], []], [[], [], [], [], [], [], [], [], [], [], [], []], [[], [], [], [], [], [], [], [], [], [], [], []]]


Warning in <TROOT::Append>: Replacing existing TH1: sec (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: sec (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: sec (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: sec (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: sec (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: sec (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: sec (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: sec (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: sec (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: sec (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: sec (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: sec (Potential memory leak).
Info in <TCanvas::Print>: pn

In [17]:
test_hist = root.TH1D("test_hist","test_hist",10,0,10)
random = root.gRandom.Rndm

func = root.TF1("func","exp(-x)*x",0,10)


for i in range(100000):
    val = random()*10
    test_hist.Fill(val,func.Eval(val)/val)

c9 = root.TCanvas("c9","c9",500,500)
test_hist.SetMarkerStyle(21)
test_hist.SetLineWidth(2)
c9.SetLogy()
test_hist.Draw("")
c9.Draw()